In [1]:
import sympy as sp
from sympy import sin, cos, pi

import pickle
import numpy as np

import matplotlib.pyplot as plt

import symbtools as st
import symbtools.modeltools as mt

In [8]:
np = 1
nq = 1
n = np + nq

x1, x2, x3, x4 = xx = sp.symbols("x1, x2, x3, x4")
ttheta = st.row_stack(x1, x2)

xdot1, xdot2 = sp.symbols("xdot1, xdot2")

params = sp.symbols('m1, m2, l1, g, s1, s2, J1, J2')
st.make_global(params)
u1 = sp.symbols("u1")
parameter_values = [(g, 9.81), (l1, .5), (J1, 1/150), (J2, 1/75), (m1, 1), (m2, 1), (s1, 0.25), (s2, 0.25), (u1, 0)]

mt.Rz(x2)

# unuit vectors
ex = sp.Matrix([1, 0])
ey = sp.Matrix([0, 1])

# coordinates of the centers of gravity and joints
S1 = mt.Rz(x1) * (-ey) * s1
G1 = mt.Rz(x1) * (-ey) * l1  # "elbow joint"
S2 = G1 + mt.Rz(x2 + x1) * (-ey) * s2

# time derivatives of the center of gravity coordinates
Sd1, Sd2 = st.col_split(st.time_deriv(st.col_stack(S1, S2), ttheta))

# kinetic energy
T_rot = (J1 * x3**2) / 2 + (J2 * (x4 + x3) ** 2) / 2
T_trans = (m1 * Sd1.T * Sd1 + m2 * Sd2.T * Sd2) / 2

T = T_rot + T_trans[0]

# potential energy
V = m1 * g * S1[1] + m2 * g * S2[1]

external_forces = [0, u1]
assert not any(external_forces[:np])
mod = mt.generate_symbolic_model(T, V, ttheta, external_forces)

mod.calc_state_eq(simplify=False)

state_eq = mod.state_eq.subs([(xdot1, x3), (xdot2, x4)])

In [13]:
h = x3
f = state_eq.subs(parameter_values)
lie = []
for i in range(4):
    lie.append(st.lie_deriv(h, f, xx, order=i))
Q = sp.Matrix(lie)

In [18]:
J = Q.jacobian(xx)
st.generic_rank(J)

/home/julius/anaconda3/envs/obs/lib/python3.9/site-packages/symbtools/core.py:3251: UserWarning: There are Float-Objects contained in the matrix. They are converted to rationals.To make sure that no harm is done, the data should be converted before passingto this function. you can use e.g. rationalize_all_numbers().
  warnings.warn(msg, UserWarning)


4

In [22]:
h = x4
f = state_eq.subs(parameter_values)
lie = []
for i in range(4):
    lie.append(st.lie_deriv(h, f, xx, order=i))
Q = sp.Matrix(lie)

In [23]:
J = Q.jacobian(xx)
st.generic_rank(J)

/home/julius/anaconda3/envs/obs/lib/python3.9/site-packages/symbtools/core.py:3251: UserWarning: There are Float-Objects contained in the matrix. They are converted to rationals.To make sure that no harm is done, the data should be converted before passingto this function. you can use e.g. rationalize_all_numbers().
  warnings.warn(msg, UserWarning)


4